In [1]:
# Make sure pip is up to date
!pip install --upgrade pip

# Install OpenCV
!pip3 install opencv-python-headless numpy

# Install PyTorch (CPU-only for Pi 4B)
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Install ultralytics (YOLO v8+)
!pip3 install ultralytics


Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://download.pytorch.org/whl/cpu, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


In [1]:
# Uncomment if you haven't installed yet
# !pip install --upgrade pip
# !pip install opencv-python-headless numpy ultralytics torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

import cv2
import numpy as np
from ultralytics import YOLO

print("Dependencies imported successfully")


Dependencies imported successfully


In [2]:
# Path to your trained model
MODEL_PATH = "trained_yolo_model/ODM-ver5.pt"

# Load YOLO model
model = YOLO(MODEL_PATH)
print("YOLO model loaded successfully")


YOLO model loaded successfully


In [5]:
# Calibration parameters
KNOWN_WIDTH_INCH = 1.0      # block width
KNOWN_DISTANCE_INCH = 12.0  # place block at this distance for calibration

cap = cv2.VideoCapture(0)
FOCAL_LENGTH = None

print("Calibration: place a single block exactly 12 inches away and press SPACE")

while True:
    ret, frame = cap.read()
    if not ret:
        continue
    
    results = model(frame)
    annotated = results[0].plot()
    
    cv2.imshow("Calibration", annotated)
    key = cv2.waitKey(1)
    
    if key & 0xFF == 32:  # SPACE key
        boxes = results[0].boxes.xywh  # x_center, y_center, w, h
        if len(boxes) > 0:
            w_pixels = boxes[0][2].item()
            FOCAL_LENGTH = (w_pixels * KNOWN_DISTANCE_INCH) / KNOWN_WIDTH_INCH
            print(f"✅ Focal length calibrated: {FOCAL_LENGTH:.2f} pixels")
            break
        else:
            print("⚠️ No block detected. Try again")
    elif key & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


Calibration: place a single block exactly 12 inches away and press SPACE

0: 480x640 (no detections), 3243.2ms
Speed: 27.8ms preprocess, 3243.2ms inference, 9.2ms postprocess per image at shape (1, 3, 480, 640)


error: OpenCV(4.12.0) /io/opencv/modules/highgui/src/window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [3]:
from IPython.display import display, clear_output
import cv2
from ultralytics import YOLO
import time

KNOWN_WIDTH_INCH = 1.0
KNOWN_DISTANCE_INCH = 12.0
FOCAL_LENGTH = None

# Load your model if not already loaded
# model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Camera not opened")


In [5]:
!pip3 install opencv-python-headless matplotlib


Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


In [6]:
from picamera2 import Picamera2
import cv2
import numpy as np

picam2 = Picamera2()
picam2.start()

while True:
    frame = picam2.capture_array()  # returns a NumPy array
    # frame is now an OpenCV BGR image
    cv2.imshow("Preview", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
picam2.close()


ModuleNotFoundError: No module named 'picamera2'

In [ ]:
from IPython.display import display, clear_output
import cv2
from ultralytics import YOLO
import time

KNOWN_WIDTH_INCH = 1.0
KNOWN_DISTANCE_INCH = 12.0
FOCAL_LENGTH = None

# Load your model if not already loaded
# model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(0)
print("Calibration: place a single block 12 inches away and press SPACE")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    start = time.time()
    results = model(frame, verbose=False)  # suppress extra logs
    end = time.time()
    
    # Annotate frame
    annotated_frame = results[0].plot()
    fps = 1 / (end - start)
    cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Convert BGR to RGB for Jupyter
    annotated_frame_rgb = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
    
    # Display in Jupyter
    clear_output(wait=True)
    display(cv2.cvtColor(annotated_frame_rgb, cv2.COLOR_BGR2RGB))

    # Check if any block detected
    if len(results[0].boxes) > 0:
        print(f"Detected {len(results[0].boxes)} block(s)")

    key = input("Press 'c' to calibrate, 'q' to quit, Enter to continue... ").lower()
    if key == 'c':
        if len(results[0].boxes) > 0:
            w_pixels = results[0].boxes.xywh[0][2].item()
            FOCAL_LENGTH = (w_pixels * KNOWN_DISTANCE_INCH) / KNOWN_WIDTH_INCH
            print(f"✅ Focal length calibrated: {FOCAL_LENGTH:.2f} pixels")
            break
        else:
            print("⚠️ No block detected, try again")
    elif key == 'q':
        break

cap.release() 
 

array([[[119, 146, 172],
        [121, 148, 174],
        [122, 152, 177],
        ...,
        [161, 171, 188],
        [160, 170, 187],
        [160, 170, 187]],

       [[120, 147, 173],
        [122, 149, 175],
        [123, 153, 178],
        ...,
        [161, 171, 188],
        [160, 170, 187],
        [160, 170, 187]],

       [[123, 150, 176],
        [124, 151, 177],
        [125, 155, 180],
        ...,
        [161, 171, 188],
        [160, 170, 187],
        [160, 170, 187]],

       ...,

       [[183, 182, 186],
        [179, 178, 182],
        [180, 179, 183],
        ...,
        [121, 111, 117],
        [113, 103, 109],
        [115, 105, 111]],

       [[165, 164, 168],
        [173, 172, 176],
        [187, 186, 190],
        ...,
        [102,  92,  98],
        [101,  91,  97],
        [103,  93,  99]],

       [[123, 122, 126],
        [145, 144, 148],
        [176, 175, 179],
        ...,
        [ 95,  85,  91],
        [ 96,  86,  92],
        [ 98,  88,  94]]

In [8]:
if FOCAL_LENGTH is not None:
    print("⚠️ Focal length not calibrated. Run calibration first.")
else:
    cap = cv2.VideoCapture(0)
    print("Starting live distance estimation. Press ESC to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        results = model(frame)
        annotated = results[0].plot()

        for box in results[0].boxes.xywh:
            w_pixels = box[2].item()
            distance_inch = (KNOWN_WIDTH_INCH * FOCAL_LENGTH) / w_pixels
            x, y = int(box[0]), int(box[1])
            cv2.putText(annotated, f"{distance_inch:.2f} in", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("YOLO Distance Estimation", annotated)
        if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
            break

    cap.release()
    cv2.destroyAllWindows()


[ WARN:0@200.049] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@200.069] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range


Starting live distance estimation. Press ESC to quit.


KeyboardInterrupt: 

In [9]:
from IPython.display import display, clear_output
import cv2

# Convert BGR -> RGB for Jupyter display
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
clear_output(wait=True)
display(frame_rgb)


error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [10]:
import cv2
from ultralytics import YOLO
import time

# --- Parameters ---
MODEL_PATH = "trained_yolo_model/ODM-ver5.pt"
KNOWN_WIDTH_INCH = 1.0        # Width of block
ASSUMED_PIXEL_WIDTH = 100.0   # Approx width of block in pixels at ~12 inches
FOCAL_LENGTH = (ASSUMED_PIXEL_WIDTH * 12.0) / KNOWN_WIDTH_INCH  # pixels

# --- Load YOLO model ---
model = YOLO(MODEL_PATH)
print("✅ YOLO model loaded")

# --- Open camera ---
cap = cv2.VideoCapture("/dev/video0")  # change to /dev/video1 if needed
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print("Starting live YOLO detection. Press ESC to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame")
        break

    start_time = time.time()
    results = model(frame, verbose=False)  # run YOLO inference
    end_time = time.time()

    annotated = results[0].plot()  # draw bounding boxes

    # Calculate distance for each detected block
    for box in results[0].boxes.xywh:
        w_pixels = box[2].item()
        distance_inch = (KNOWN_WIDTH_INCH * FOCAL_LENGTH) / w_pixels
        x, y = int(box[0]), int(box[1])
        cv2.putText(annotated, f"{distance_inch:.2f} in", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show FPS on frame
    fps = 1 / (end_time - start_time)
    cv2.putText(annotated, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Display
    cv2.imshow("YOLO Distance Estimation", annotated)

    # Stop with ESC
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


✅ YOLO model loaded
Starting live YOLO detection. Press ESC to quit.


error: OpenCV(4.12.0) /io/opencv/modules/highgui/src/window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [11]:
from ultralytics import YOLO
from IPython.display import display, clear_output
import cv2
import time

# --- Parameters ---
MODEL_PATH = "trained_yolo_model/ODM-ver5.pt"
KNOWN_WIDTH_INCH = 1.0
ASSUMED_PIXEL_WIDTH = 100.0
FOCAL_LENGTH = (ASSUMED_PIXEL_WIDTH * 12.0) / KNOWN_WIDTH_INCH

# --- Load YOLO model ---
model = YOLO(MODEL_PATH)
print("✅ YOLO model loaded")

# --- Open camera ---
cap = cv2.VideoCapture("/dev/video0")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print("Starting live YOLO detection. Press 'q' in input to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame")
        break

    start_time = time.time()
    results = model(frame, verbose=False)
    end_time = time.time()

    annotated = results[0].plot()

    # Calculate distance for each detected block
    for box in results[0].boxes.xywh:
        w_pixels = box[2].item()
        distance_inch = (KNOWN_WIDTH_INCH * FOCAL_LENGTH) / w_pixels
        x, y = int(box[0]), int(box[1])
        cv2.putText(annotated, f"{distance_inch:.2f} in", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    fps = 1 / (end_time - start_time)
    cv2.putText(annotated, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Convert for Jupyter display
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    clear_output(wait=True)
    display(annotated_rgb)

    key = input("Press Enter to continue or 'q' to quit: ").lower()
    if key == 'q':
        break

cap.release()


✅ YOLO model loaded
Starting live YOLO detection. Press 'q' in input to quit.
❌ Failed to grab frame


In [12]:
import cv2
from ultralytics import YOLO
import time

# --- Parameters ---
MODEL_PATH = "trained_yolo_model/ODM-ver5.pt"
KNOWN_WIDTH_INCH = 1.0
ASSUMED_PIXEL_WIDTH = 100.0
FOCAL_LENGTH = (ASSUMED_PIXEL_WIDTH * 12.0) / KNOWN_WIDTH_INCH

# --- Load YOLO model ---
model = YOLO(MODEL_PATH)
print("✅ YOLO model loaded")

# --- Open camera ---
cap = cv2.VideoCapture("/dev/video0")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print("Running YOLO + distance estimation (headless). Press Ctrl+C to stop.")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Failed to grab frame")
            break

        start_time = time.time()
        results = model(frame, verbose=False)
        end_time = time.time()

        # Calculate distance for each detected block
        distances = []
        for box in results[0].boxes.xywh:
            w_pixels = box[2].item()
            distance_inch = (KNOWN_WIDTH_INCH * FOCAL_LENGTH) / w_pixels
            distances.append(distance_inch)

        # Print results
        if distances:
            print(f"Detected {len(distances)} block(s), distances (inches): {[round(d, 2) for d in distances]}")

        fps = 1 / (end_time - start_time)
        print(f"FPS: {fps:.2f}")

except KeyboardInterrupt:
    print("Stopped by user")

finally:
    cap.release()
    print("Camera released")


✅ YOLO model loaded
Running YOLO + distance estimation (headless). Press Ctrl+C to stop.
FPS: 0.34
FPS: 0.37
FPS: 0.40
FPS: 0.43
FPS: 0.41
FPS: 0.41
FPS: 0.41
FPS: 0.42
FPS: 0.42
FPS: 0.42
FPS: 0.39
FPS: 0.41
FPS: 0.39
FPS: 0.39
Stopped by user
Camera released


In [13]:
import cv2
from ultralytics import YOLO
import time

# --- Parameters ---
MODEL_PATH = "trained_yolo_model/ODM-ver5.pt"
KNOWN_WIDTH_INCH = 1.0
ASSUMED_PIXEL_WIDTH = 100.0
FOCAL_LENGTH = (ASSUMED_PIXEL_WIDTH * 12.0) / KNOWN_WIDTH_INCH

# --- Load YOLO model ---
model = YOLO(MODEL_PATH)
print("✅ YOLO model loaded")

# --- Open camera ---
cap = cv2.VideoCapture("/dev/video0")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print("Running YOLO + distance estimation headless for 10 seconds...")

start_time_total = time.time()
snapshot_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Failed to grab frame")
            break

        start_time = time.time()
        results = model(frame, verbose=False)
        end_time = time.time()

        # Calculate distance for each detected block
        distances = []
        for box in results[0].boxes.xywh:
            w_pixels = box[2].item()
            distance_inch = (KNOWN_WIDTH_INCH * FOCAL_LENGTH) / w_pixels
            distances.append(distance_inch)

        # Print results
        if distances:
            print(f"Detected {len(distances)} block(s), distances (inches): {[round(d,2) for d in distances]}")

        fps = 1 / (end_time - start_time)
        print(f"FPS: {fps:.2f}")

        # Save snapshot every 5 frames
        snapshot_count += 1
        if snapshot_count % 5 == 0:
            cv2.imwrite(f"frame_{snapshot_count}.jpg", frame)
            print(f"Saved frame_{snapshot_count}.jpg")

        # Stop after 10 seconds
        if time.time() - start_time_total >= 10:
            print("⏱ 10 seconds elapsed, stopping...")
            break

except KeyboardInterrupt:
    print("Stopped by user")

finally:
    cap.release()
    print("Camera released")


✅ YOLO model loaded
Running YOLO + distance estimation headless for 10 seconds...
Detected 2 block(s), distances (inches): [17.71, 16.47]
FPS: 0.27
Detected 2 block(s), distances (inches): [17.73, 16.74]
FPS: 0.33
Detected 2 block(s), distances (inches): [17.62, 16.43]
FPS: 0.40
Detected 2 block(s), distances (inches): [17.64, 16.38]
FPS: 0.39
⏱ 10 seconds elapsed, stopping...
Camera released


In [14]:
# Notebook-friendly installation of Picamera2 and dependencies
!sudo apt update
!sudo apt install -y python3-picamera2 python3-libcamera python3-numpy


Hit:1 http://deb.debian.org/debian bookworm InRelease
Hit:2 http://deb.debian.org/debian-security bookworm-security InRelease
Get:3 http://deb.debian.org/debian bookworm-updates InRelease [55.4 kB]
Hit:4 http://archive.raspberrypi.com/debian bookworm InRelease             
Fetched 55.4 kB in 3s (16.4 kB/s)         
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
307 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-numpy is already the newest version (1:1.24.2-1+deb12u1).
The following packages were automatically installed and are no longer required:
  avahi-utils gir1.2-handy-1 gir1.2-packagekitglib-1.0 gir1.2-polkit-1.0
  libc++1-16 libc++abi1-16 libcamera0.3 libunwind-16 libwlroots12
  lxplug-network python3-v4l2
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libb2-

In [4]:
import cv2
from ultralytics import YOLO
import time
import os

# --- Parameters ---
MODEL_PATH = "trained_yolo_model/ODM-ver5.pt"
KNOWN_WIDTH_INCH = 1.0            # Width of block in inches
ASSUMED_PIXEL_WIDTH = 100.0       # Approx width of block in pixels at ~12 inches
FOCAL_LENGTH = (ASSUMED_PIXEL_WIDTH * 12.0) / KNOWN_WIDTH_INCH

SAVE_DIR = "snapshots"
os.makedirs(SAVE_DIR, exist_ok=True)

# --- Load YOLO model ---
model = YOLO(MODEL_PATH)
print("✅ YOLO model loaded")

# --- Open camera ---
cap = cv2.VideoCapture(0)  # 0 = default camera
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    raise RuntimeError("❌ Could not open camera. Make sure a camera is connected.")

print("Running YOLO + distance estimation headless for 10 seconds...")

start_time_total = time.time()
snapshot_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            print("❌ Failed to grab frame")
            continue

        # Run YOLO
        start_time = time.time()
        results = model(frame, verbose=False)
        end_time = time.time()

        # Calculate distances
        distances = []
        for box in results[0].boxes.xywh:
            w_pixels = box[2].item()
            distance_inch = (KNOWN_WIDTH_INCH * FOCAL_LENGTH) / w_pixels
            distances.append(distance_inch)

        # Print detected distances
        if distances:
            print(f"Detected {len(distances)} block(s), distances (inches): {[round(d,2) for d in distances]}")

        fps = 1 / (end_time - start_time)
        print(f"FPS: {fps:.2f}")

        # Save snapshot every 5 frames
        snapshot_count += 1
        if snapshot_count % 5 == 0:
            filepath = os.path.join(SAVE_DIR, f"frame_{snapshot_count}.jpg")
            success = cv2.imwrite(filepath, frame)
            if success:
                print(f"✅ Saved {filepath}")
            else:
                print(f"❌ Failed to save {filepath}")

        # Stop after 10 seconds
        if time.time() - start_time_total >= 10:
            print("⏱ 10 seconds elapsed, stopping...")
            break

except KeyboardInterrupt:
    print("Stopped by user")

finally:
    cap.release()
    print("Camera released")
    print("Snapshots saved in folder:", os.path.abspath(SAVE_DIR))


✅ YOLO model loaded
Running YOLO + distance estimation headless for 10 seconds...
FPS: 0.36
Detected 1 block(s), distances (inches): [19.55]
FPS: 0.42
FPS: 0.47
FPS: 0.49
FPS: 0.37
✅ Saved snapshots/frame_5.jpg
⏱ 10 seconds elapsed, stopping...
Camera released
Snapshots saved in folder: /home/clopezgarcia2/Desktop/roboforge/robofore-vision/snapshots
